# Prepare the data

In [547]:
import gdown
import os

# Download URL, the shakespeare.txt
url = f'https://drive.google.com/uc?id=1O4PZ8wOpp6yecoy8tMuVEIFS7XgyRJy9'

data_path = '../data'
text_path = f'{data_path}/shakespeare.txt'

if not os.path.exists(data_path):
    os.makedirs(data_path)

if not os.path.exists(text_path):
  gdown.download(url, text_path, quiet=False)


In [548]:
import re

with open(text_path) as f:
  text = f.read()
  
text = re.sub(r'\d+', '', text)

print(f"lenth of the text {len(text)}")

lenth of the text 5433453


In [549]:
print(text[:1000])

  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep sunken eyes,
  Were an all-ea

In [550]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocab size : {len(chars)}")
print("".join(chars))

vocab size : 74

 !"&'(),-.:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}


# Prepare tokenizer

In [551]:
# import tiktoken
# tokenizer = tiktoken.get_encoding('gpt2')
# tokens = tokenizer.encode(text)
# print(f"total tokens {len(tokens)}")
# print("decode result of \"hello world.\"", tokenizer.decode([31373, 995]))

In [552]:
class SimpleTokenizer:
  def __init__(self, text):
    self.chars = sorted(list(set(text)))
    self.token2id = {c : i for i, c in enumerate(chars)}
    self.id2token = {i : c for i, c in enumerate(chars)}
    
  def encode(self, text):
    return [self.token2id[c] for c in text]
  
  def decode(self, token_ids):
    return "".join([self.id2token[token_id] for token_id in token_ids])
  

In [553]:
tokenizer = SimpleTokenizer(text)
vocab_size = len(tokenizer.chars)
print(
  tokenizer.encode("Hello"),
  tokenizer.decode([23, 50, 57, 57, 60]),
  sep='\n')

[23, 50, 57, 57, 60]
Hello


# Prepare Data for torch

In [554]:
import torch

data = torch.tensor(tokenizer.encode(text), dtype = torch.long) # torch.long can be used as index directly
print(data, data.shape, data.dtype)


tensor([ 1,  1, 21,  ..., 29, 19,  0]) torch.Size([5433453]) torch.int64


In [555]:
train_data_size = int(data.shape[0] * 0.9)
train_data = data[:train_data_size].detach()
val_data = data[train_data_size:].detach()

In [556]:
from torch.utils.data import Dataset, DataLoader
class SimpleDataset(Dataset):
  def __init__(self, data, block_size = 8):
    self.data = data
    self.block_size = block_size
    
  def __len__(self):
    return len(self.data) - self.block_size

  def __getitem__(self, idx):
    x = self.data[idx: idx + self.block_size]
    y = self.data[idx + self.block_size]
    return x, y

In [557]:
train_dataset = SimpleDataset(train_data)
val_dataset = SimpleDataset(val_data)

In [558]:
print(len(train_dataset), len(val_dataset))

4890099 543338


In [559]:
data[:9]

tensor([ 1,  1, 21, 63, 60, 58,  1, 51, 46])

In [560]:
train_dataset[0]

(tensor([ 1,  1, 21, 63, 60, 58,  1, 51]), tensor(46))

In [561]:
import numpy as np
class SimpleDataloader(DataLoader):
  def __init__(self, dataset, batch_size=4, shuffle=True, **kwargs):
    super().__init__(dataset, batch_size=batch_size, shuffle=shuffle, **kwargs)
    self.shuffle = shuffle
  def __iter__(self):
    dataset_size = len(self.dataset)
    indices = np.arange(dataset_size)
    if self.shuffle:
        np.random.shuffle(indices)

    for start_idx in range(0, dataset_size - self.batch_size + 1, self.batch_size):
        batch_indices = indices[start_idx:start_idx + self.batch_size]
        yield (torch.stack([self.dataset[i][0] for i in batch_indices]),
              torch.stack([self.dataset[i][1] for i in batch_indices]))

In [562]:
torch.manual_seed(0)
train_dataloader = SimpleDataloader(train_dataset)
val_dataloader = SimpleDataloader(val_dataset)

In [563]:
for i, batch in enumerate(train_dataloader):
  print(batch)
  if i == 3:
    break

(tensor([[70, 60, 66, 63,  1, 60, 68, 59],
        [59, 49,  1, 53, 50, 63, 50,  8],
        [54, 57, 60, 64, 60, 61, 53, 50],
        [60, 51,  1, 53, 50, 46, 67, 50]]), tensor([12,  1, 63, 59]))
(tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [47, 66, 65,  1, 46, 59,  1, 46],
        [63, 49,  8,  1, 54, 64,  1, 67],
        [53, 50, 54, 63,  1, 47, 50, 64]]), tensor([ 1, 57, 46, 65]))
(tensor([[ 1,  5, 65, 54, 64,  1, 58, 60],
        [ 1, 16, 64,  1, 58, 66, 48, 53],
        [12,  1, 46, 59, 49,  1, 65, 53],
        [50, 63, 70, 10,  0,  1,  1,  1]]), tensor([64,  1, 60,  1]))
(tensor([[64,  1, 68, 54, 57, 57,  1, 46],
        [65, 53, 50,  1, 59, 60, 47, 57],
        [57, 46, 65, 50, 64, 65,  1, 61],
        [59, 54, 59, 52, 64,  8,  1, 64]]), tensor([59, 50, 46, 54]))


# Transformer GPT

In [564]:
# Model Configuration
VOCAB_SIZE = 74 # Should be set according to the tokenizer
EMBED_DIM = 12

In [565]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGPT(nn.Module):
  def __init__(self):
    super().__init__()
    # (B, T)
    self.embedding = nn.Embedding(VOCAB_SIZE, VOCAB_SIZE) #(B, T, T)
    

  def forward(self, x):
    """
    x should be in the form of (B, T)
    The out out is (B, Vocab_size), indicating the probability
    """
    y = self.embedding(x)
    y = y[:, -1,:]
    y = F.softmax(y, 1)
    return y
  
  def generate(self, x, max_tokens = 30):
    """
    x (B, T)

    ### returns
    y (B, T + max_tokens)

    ### Warning
    Because we do not have an EOF, so it will generate max_tokens actually
    """
    for _ in range(max_tokens):
      # x shape (B, T)
      y = self.forward(x) # y shape (B, vocab_size)

      
      next_idx = torch.multinomial(y, 1)  # next_idx shape(B, 1)
      x = torch.cat([x, next_idx], dim = 1)

    return x

In [566]:
gpt = SimpleGPT()

In [567]:
input = torch.zeros(1, 1, dtype=torch.long)
output = gpt.generate(input)
print(tokenizer.decode(output[0].tolist()))


xz|
ns
W>-OHO[KO}}AiGFgfF eb>Z
